In [1]:
import os
import random
import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn.functional import normalize
from math import sqrt
from detorch import DE, Policy, Strategy
from detorch.config import default_config, Config
from typing import Type
import numpy as np
import sys
import matplotlib.pyplot as plt
sys.path.append('..')
sys.path.append('viz')
from optimneuralts import DENeuralTSDiag, LenientDENeuralTSDiag
from networks import NetworkDropout
from utils import do_gradient_optim
import viz_config
import logging

logging.basicConfig(level=logging.INFO)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
config = Config(default_config)
bounds = [-2.5, 1.5]
theta = torch.Tensor([0, 3, -2, -4, 1, 1]).to(device)
d = 1
torch.set_default_tensor_type("torch.cuda.FloatTensor")

%load_ext autoreload
%autoreload 2

# Classes

In [2]:
class PullPolicy(Policy):
    def __init__(self, eval_fn):
        super().__init__()
        self.point = torch.FloatTensor(1).uniform_(*bounds).to(device)
        self.params = nn.Parameter(
            self.point, requires_grad=False
        )
        self.eval_fn = eval_fn
        self.ucb = None

    def evaluate(self):
        self.transform()
        ucb, activation_grad, _, _ = self.eval_fn(self.point)
        ucb = ucb.detach().item()
        self.activation_grad = activation_grad
        self.ucb = ucb
        # logging.info(self.point)
        # logging.info(ucb)
        return ucb

    def transform(self):
        self.point = torch.clip(self.params, *bounds).to(device)
        self.params = nn.Parameter(
            self.point, requires_grad=False
        )
        # return generate_feature_vector_from_point(self.point)


class DEConfig:
    n_step: int = 3
    population_size: int = 60
    differential_weight: float = 0.8
    crossover_probability: float = 0.9
    strategy: Strategy = Strategy.best1bin
    seed: int = "does not matter"


# Utility

In [3]:
def compute_jaccard(found_solution, true_solution):
    found_sol_list = found_solution.tolist()
    true_sol_list = true_solution.tolist()

    n_in_inter = 0

    for vec in found_sol_list:
        n_in_inter += vec in true_sol_list

    return n_in_inter / (len(found_solution) + len(true_solution) - n_in_inter), n_in_inter

def make_deterministic(seed):
    # PyTorch
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

    # Numpy
    np.random.seed(seed)

    # Built-in Python
    random.seed(seed)

def project_point(point):
    return torch.tensor([1, point, point ** 2, point ** 3, point ** 4, point ** 5]).to(device)

def reward_fn(point, add_noise=True):
    noise = torch.normal(0, 1, (1,)).item()

    vec = project_point(point)
    value = theta @ vec.T
    
    return value + add_noise * noise


def gen_warmup_vecs_and_rewards(n_warmup):
    vecs = torch.tensor([])
    rewards = torch.tensor([])
    for i in range(n_warmup):
        point = torch.FloatTensor(1).uniform_(bounds[0], bounds[1]).to(device)
        reward = torch.tensor([reward_fn(point)])

        vecs = torch.cat((vecs, point))
        rewards = torch.cat((rewards, reward))
    vecs = vecs.view((n_warmup, -1))
    rewards = rewards.view((n_warmup, -1))
    return vecs, rewards


def plot_estimate(agent, trial, fn=None, title=""):
    n_points = 1000
    x = np.linspace(-2.5, 1.5, n_points)
    x_vec = []
    y = []
    for x_point in x:
        y.append(reward_fn(float(x_point), add_noise=False).cpu().numpy())

    y_pred = []
    cbs = []
    ucbs = []
    x_tns = torch.from_numpy(x)
    x_tns = x_tns.view(n_points, 1).float()

    for point in x_tns:
        sample, _, activ, cb = agent.get_sample(point.to(device))
        y_pred.append(activ)
        cbs.append(3 * cb)
        ucbs.append(sample)

    y_pred = np.array(y_pred)
    cbs = np.array(cbs)

    point_played = agent.train_dataset.features.squeeze(0).cpu().numpy()
    rewards_rec = agent.train_dataset.rewards.squeeze(0).cpu().numpy()
    n_played = point_played.shape[0]

    # plt.ylim(-20, 10)
    # plt.ylim(-25, 10)
    plt.plot(x, y, color="tab:blue", label="Vraie fonction")
    plt.plot(x, y_pred, color="tab:orange", label="Estimation de la fonction")
    # plt.fill_between(x, y_pred, ucbs, color='tab:red', alpha=0.3)
    plt.fill_between(
        x, y_pred + cbs, y_pred - cbs, alpha=0.3, color="tab:orange", zorder=-1, label="Intervalle de confiance",
    )
    plt.plot(
        x,
        [0] * n_points,
        color="black",
        linestyle="dashed",
        label="Seuil bonne/mauvaise action",
    )
    plt.scatter(
        point_played[:n_played],
        rewards_rec[:n_played],
        color="black",
        alpha=0.5,
        label="Points joués précédemment",
    )
    plt.scatter(
        point_played[-1], rewards_rec[-1], color="green", label="Dernier point joué"
    )

    plt.title(title)
    
    plt.xlabel("$x$")
    plt.ylabel("$y$")


    plt.legend()
    if fn is None:
        filename = f"viz/images/exp_poly/regTS_{n_trials}_trials_expl_{exploration_mult}_trial_{trial}.png"
    else:
        filename = f"viz/images/exp_poly/{fn}.png"
    plt.savefig(filename)

    plt.clf()


def find_best_member(eval_fn, de_config, seed):
    de_config.seed = seed
    config = Config(default_config)

    @config("policy")
    class PolicyConfig:
        policy: Type[Policy] = PullPolicy
        eval_fn: object = agent.get_sample

    config("de")(de_config)

    de = DE(config)
    de.train()

    return de.population[de.current_best]


def gen_mask(example_vec, agent, account=False):
    # Mask for 1 vec
    eval_mask = (
        torch.distributions.Bernoulli(
            torch.full_like(example_vec, (1 - dropout_rate))
        ).sample()
        / (1 - dropout_rate)
    ).to(device)

    n_obs = list(agent.train_dataset.features.shape)[0]

    if account:
        n_obs + 1

    # Mask for multiple vecs
    training_mask = (
        torch.distributions.Bernoulli(
            torch.full((n_obs, agent.net.hidden_size), (1 - dropout_rate))
        ).sample()
        / (1 - dropout_rate)
    ).to(device)

    return eval_mask, training_mask



# Visualize 1 run

In [4]:
run = 42
make_deterministic(run)

n_trials = 500
dropout_rate = False
width = 100
net = NetworkDropout(d, width,dropout=dropout_rate).to(device)
reg = 1
exploration_mult = 1
delay = 0
reward_fn = reward_fn
style="ts"
sampletype="r"
max_n_steps = 100 
agent = DENeuralTSDiag(net, nu=exploration_mult, lambda_=reg, style=style, sampletype=sampletype)
# agent = LenientDENeuralTSDiag(net, nu=exploration_mult, lambda_=reg, reward_sample_thresholds=[float('-inf'), 0])


In [5]:
vecs, rewards = gen_warmup_vecs_and_rewards(10)
vecs, rewards = vecs.to(device), rewards.to(device)

example_vec = vecs[0]

/tmp/ipykernel_3419420/1693466289.py:32: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matricesor `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2318.)
  value = theta @ vec.T


In [6]:
agent.train_dataset.set_(vecs, rewards)

for vec in vecs:
    activ, grad = agent.compute_activation_and_grad(vec)
    agent.U += grad * grad

agent.net.eval()
agent.train(max_n_steps, patience=max_n_steps, lds=False)
agent.net.train()


NetworkDropout(
  (fc1): Linear(in_features=1, out_features=100, bias=True)
  (activate): ReLU()
  (fc2): Linear(in_features=100, out_features=1, bias=True)
  (dropout): Identity()
)

In [7]:
plot_estimate(agent, 0,  f'grad_{style}_expl_{exploration_mult}_{sampletype}_run{run}_trial_0')
agent.net.train()


NetworkDropout(
  (fc1): Linear(in_features=1, out_features=100, bias=True)
  (activate): ReLU()
  (fc2): Linear(in_features=100, out_features=1, bias=True)
  (dropout): Identity()
)

<Figure size 980x700 with 0 Axes>

### Train 1 run

In [8]:
for i in range(n_trials):
    agent.net.train()
    a_t, idx, best_member_grad = do_gradient_optim(
        agent, 3 * 60, x, lr=1e-2
    )
    agent.net.eval()

    r_t = reward_fn(a_t).unsqueeze(0).unsqueeze(0)

    logging.info(f"trial: {i}")

    agent.U += best_member_grad * best_member_grad

    agent.train_dataset.add(a_t, r_t)

    agent.train(max_n_steps, patience=max_n_steps, lds=False)
    if (i + 1) % 100 == 0:
        plot_estimate(
            agent,
            0,
            f"grad_{style}_expl_{exploration_mult}_{sampletype}_run{run}_trial_{i+1}"
        )


NameError: name 'x' is not defined

# Train all runs, for all algos and every exploration

In [9]:
x_arr = np.linspace(-2.5, 1.5, 1000)
x_arr = x_arr.reshape(1000, 1)
x = torch.from_numpy(x_arr).to(device).float()
y = []
for point in x:
    y.append(reward_fn(point, add_noise=False).cpu().numpy())

y = np.array(y)
true_sol = x[np.where(y >= 0)[0]].to(device).float()
n_true_sol = len(true_sol)
logging.info(n_true_sol)
n_sigmas = 3

INFO:root:419


In [12]:
metrics_dict = {}
algos = ["UCB", "regTS", "lenientTS"]
exploration_mults = [1, 10]
logging.info(metrics_dict)
max_n_steps = 10
for algo in algos:
    metrics_dict[algo] = {}
    for exploration_mult in exploration_mults:
        metrics_dict[algo][str(exploration_mult)] = {}
        metrics_dict[algo][str(exploration_mult)]["jaccards"] = []
        metrics_dict[algo][str(exploration_mult)]["percent_inter"] = []
        metrics_dict[algo][str(exploration_mult)]["percent_found"] = []
        metrics_dict[algo][str(exploration_mult)]["fails"] = 0

        b = 0

        for i in range(b, 50):
            logging.info(f"at algo: {algo}, expl_mult: {exploration_mult}, run: {i}")
            make_deterministic(seed=i)
            vecs, rewards = gen_warmup_vecs_and_rewards(10)
            vecs, rewards = vecs.to(device), rewards.to(device)
            n_trials = 500
            width = 100
            net = NetworkDropout(d, width).to(device)
            reg = 1
            delay = 0
            reward_fn = reward_fn
            de_config = DEConfig
            de_policy = PullPolicy
            lr = 1e-2

            if algo == "UCB":
                agent = DENeuralTSDiag(
                    net, nu=exploration_mult, lambda_=reg, style="ucb"
                )
            elif algo == "regTS":
                agent = DENeuralTSDiag(
                    net, nu=exploration_mult, lambda_=reg, style="ts"
                )
            elif algo == "lenientTS":
                agent = LenientDENeuralTSDiag(
                    [float("-inf"), 0],
                    net=net,
                    nu=exploration_mult,
                    lambda_=reg,
                )

            vecs, rewards = gen_warmup_vecs_and_rewards(10)
            vecs, rewards = vecs.to(device), rewards.to(device)
            # Warmup
            for vec in vecs:
                activ, grad = agent.compute_activation_and_grad(vec)
                agent.U += grad * grad

            agent.train_dataset.set_(vecs, rewards)
            agent.net.eval()
            agent.train(max_n_steps, patience=max_n_steps, lds=False)

            # Playing
            for j in range(n_trials):
                agent.net.train()
                a_t, idx, best_member_grad = do_gradient_optim(agent, 3 * 60, x, lr=1e-2)
                agent.net.eval()
                r_t = reward_fn(a_t).unsqueeze(0).unsqueeze(0)

                agent.U += best_member_grad * best_member_grad

                agent.train_dataset.add(a_t, r_t)

                agent.train(max_n_steps, patience=max_n_steps, lds=False)
            # Stop using mask in evaluation
            agent.net.eval()
            sol, _, _ = agent.find_solution_in_vecs(x, 0, n_sigmas=n_sigmas)
            sol = sol.to(device)
            n_sol = len(sol)

            jaccard, n_inter = compute_jaccard(sol, true_sol)
            percent_found = n_inter / n_true_sol
            if n_sol == 0:
                percent_inter = 0
            else:
                percent_inter = n_inter / n_sol

            metrics_dict[algo][str(exploration_mult)]["jaccards"].append(jaccard)
            metrics_dict[algo][str(exploration_mult)]["percent_inter"].append(
                percent_inter
            )
            metrics_dict[algo][str(exploration_mult)]["percent_found"].append(
                percent_found
            )

            if n_sol == 0:
                logging.info(f"Found no solution for run {i}")
                metrics_dict[algo][str(exploration_mult)]["fails"] += 1

                plot_estimate(
                    agent,
                    n_trials,
                    fn=f"grad_no_sol_{algo}_expl_{exploration_mult}_100_trials_seed_{i}",
                )

            logging.info(
                f"jaccard: {jaccard}, percent_inter: {percent_inter}, percent_found: {percent_found}"
            )

torch.save(metrics_dict, f"metrics_dict_exp_synth_{max_n_steps}_basic.pth")


INFO:root:{}
INFO:root:at algo: UCB, expl_mult: 1, run: 0
INFO:root:jaccard: 0.16706443914081145, percent_inter: 1.0, percent_found: 0.16706443914081145
INFO:root:at algo: UCB, expl_mult: 1, run: 1
INFO:root:jaccard: 0.11455847255369929, percent_inter: 1.0, percent_found: 0.11455847255369929
INFO:root:at algo: UCB, expl_mult: 1, run: 2
INFO:root:Found no solution for run 2
INFO:root:jaccard: 0.0, percent_inter: 0, percent_found: 0.0
INFO:root:at algo: UCB, expl_mult: 1, run: 3
INFO:root:jaccard: 0.16467780429594273, percent_inter: 1.0, percent_found: 0.16467780429594273
INFO:root:at algo: UCB, expl_mult: 1, run: 4
INFO:root:jaccard: 0.050119331742243436, percent_inter: 1.0, percent_found: 0.050119331742243436
INFO:root:at algo: UCB, expl_mult: 1, run: 5
INFO:root:Found no solution for run 5
INFO:root:jaccard: 0.0, percent_inter: 0, percent_found: 0.0
INFO:root:at algo: UCB, expl_mult: 1, run: 6
INFO:root:jaccard: 0.021479713603818614, percent_inter: 1.0, percent_found: 0.02147971360381

RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

<Figure size 980x700 with 0 Axes>

In [13]:
for algo in metrics_dict.keys():
    logging.info(f"Algo: {algo}")
    for expl_mult in metrics_dict[algo].keys():
        logging.info(f'Expl mult: {expl_mult}')
        for metric in  metrics_dict[algo][expl_mult].keys():
            logging.info(f'Metric: {metric}')
            logging.info(f"mean: {np.mean(metrics_dict[algo][expl_mult][metric])} +- {np.std(metrics_dict[algo][expl_mult][metric])} ")
            logging.info(f"interval:  [{np.min(metrics_dict[algo][expl_mult][metric])}, {np.max(metrics_dict[algo][expl_mult][metric])}]")
            logging.info("============================================")
        # if 0 in metrics_dict[algo][expl_mult]:
        #     logging.info(f'rerun {metrics_dict[algo][expl_mult][metric].index(0)} with plotting')
    
    logging.info("============================================")
    

INFO:root:Algo: UCB
INFO:root:Expl mult: 1
INFO:root:Metric: jaccards
INFO:root:mean: 0.07069212410501194 +- 0.07150493417149824 
INFO:root:interval:  [0.0, 0.2386634844868735]
INFO:root:============================================
INFO:root:Metric: percent_inter
INFO:root:mean: 0.66 +- 0.4737087712930804 
INFO:root:interval:  [0.0, 1.0]
INFO:root:============================================
INFO:root:Metric: percent_found
INFO:root:mean: 0.07069212410501194 +- 0.07150493417149824 
INFO:root:interval:  [0.0, 0.2386634844868735]
INFO:root:============================================
INFO:root:Metric: fails
INFO:root:mean: 17.0 +- 0.0 
INFO:root:interval:  [17, 17]
INFO:root:============================================
INFO:root:Expl mult: 10
INFO:root:Metric: jaccards
INFO:root:mean: 0.15164677804295942 +- 0.07601379152542324 
INFO:root:interval:  [0.0, 0.2840095465393795]
INFO:root:============================================
INFO:root:Metric: percent_inter
INFO:root:mean: 0.88 +- 0.32

ValueError: zero-size array to reduction operation minimum which has no identity

## Test avec weight decay

In [14]:
metrics_dict = {}
# algos = ["UCB", "regTS", "lenientTS"]
algos = ["UCB", "regTS"]
exploration_mults = [1, 10]
logging.info(metrics_dict)
max_n_steps = 10
for algo in algos:
    metrics_dict[algo] = {}
    for exploration_mult in exploration_mults:
        metrics_dict[algo][str(exploration_mult)] = {}
        metrics_dict[algo][str(exploration_mult)]["jaccards"] = []
        metrics_dict[algo][str(exploration_mult)]["percent_inter"] = []
        metrics_dict[algo][str(exploration_mult)]["percent_found"] = []
        metrics_dict[algo][str(exploration_mult)]["fails"] = 0

        b = 0

        for i in range(b, 50):
            logging.info(f"at algo: {algo}, expl_mult: {exploration_mult}, run: {i}")
            make_deterministic(seed=i)
            vecs, rewards = gen_warmup_vecs_and_rewards(10)
            vecs, rewards = vecs.to(device), rewards.to(device)
            n_trials = 500
            width = 100
            net = NetworkDropout(d, width).to(device)
            reg = 1
            reward_fn = reward_fn
            de_config = DEConfig
            de_policy = PullPolicy
            lr = 1e-2

            if algo == "UCB":
                agent = DENeuralTSDiag(
                    net, nu=exploration_mult, lambda_=reg, style="ucb"
                )
            elif algo == "regTS":
                agent = DENeuralTSDiag(net, nu=exploration_mult, lambda_=reg, style="ts")
            elif algo == "lenientTS":
                agent = LenientDENeuralTSDiag(
                    [float("-inf"), 0],
                    net=net,
                    nu=exploration_mult,
                    lambda_=reg,
                )

            vecs, rewards = gen_warmup_vecs_and_rewards(10)
            vecs, rewards = vecs.to(device), rewards.to(device)
            agent.train_dataset.set_(vecs, rewards)

            # Warmup
            for vec in vecs:
                activ, grad = agent.compute_activation_and_grad(vec)
                agent.U += grad * grad

            agent.net.eval()
            agent.train(max_n_steps, patience=max_n_steps, lds=False, use_decay=True)
            # Playing
            for j in range(n_trials):
                agent.net.train()
                a_t, idx, best_member_grad = do_gradient_optim(agent, 60, x, lr=1e-2)
                agent.net.eval()
                r_t = reward_fn(a_t).unsqueeze(0).unsqueeze(0)

                agent.U += best_member_grad * best_member_grad

                agent.train_dataset.add(a_t, r_t)

                agent.train(max_n_steps, patience=max_n_steps, lds=False, use_decay=True)

            # Stop using mask in evaluation
            sol, _, _ = agent.find_solution_in_vecs(x, 0, n_sigmas=n_sigmas)
            sol = sol.to(device)
            n_sol = len(sol)

            jaccard, n_inter = compute_jaccard(sol, true_sol)
            percent_found = n_inter / n_true_sol
            if n_sol == 0:
                percent_inter = 0
            else:
                percent_inter = n_inter / n_sol

            metrics_dict[algo][str(exploration_mult)]["jaccards"].append(jaccard)
            metrics_dict[algo][str(exploration_mult)]["percent_inter"].append(
                percent_inter
            )
            metrics_dict[algo][str(exploration_mult)]["percent_found"].append(
                percent_found
            )

            if n_sol == 0:
                logging.info(f"Found no solution for run {i}")
                plot_estimate(
                    agent,
                    n_trials,
                    fn=f"grad_no_sol_{algo}_decay_expl_{exploration_mult}_100_trials_seed_{i}",
                )
                metrics_dict[algo][str(exploration_mult)]["fails"] += 1


            logging.info(
                f"jaccard: {jaccard}, percent_inter: {percent_inter}, percent_found: {percent_found}"
            )

torch.save(metrics_dict, f"metrics_dict_exp_synth_{max_n_steps}_decay.pth")


INFO:root:{}
INFO:root:at algo: UCB, expl_mult: 1, run: 0
INFO:root:jaccard: 0.1431980906921241, percent_inter: 1.0, percent_found: 0.1431980906921241
INFO:root:at algo: UCB, expl_mult: 1, run: 1
INFO:root:jaccard: 0.12649164677804295, percent_inter: 1.0, percent_found: 0.12649164677804295
INFO:root:at algo: UCB, expl_mult: 1, run: 2
INFO:root:jaccard: 0.17899761336515513, percent_inter: 1.0, percent_found: 0.17899761336515513
INFO:root:at algo: UCB, expl_mult: 1, run: 3
INFO:root:jaccard: 0.19809069212410502, percent_inter: 1.0, percent_found: 0.19809069212410502
INFO:root:at algo: UCB, expl_mult: 1, run: 4
INFO:root:jaccard: 0.15990453460620524, percent_inter: 1.0, percent_found: 0.15990453460620524
INFO:root:at algo: UCB, expl_mult: 1, run: 5
INFO:root:jaccard: 0.1360381861575179, percent_inter: 1.0, percent_found: 0.1360381861575179
INFO:root:at algo: UCB, expl_mult: 1, run: 6
INFO:root:jaccard: 0.21241050119331742, percent_inter: 1.0, percent_found: 0.21241050119331742
INFO:root:a

In [15]:
for algo in metrics_dict.keys():
    logging.info(f"Algo: {algo}")
    for expl_mult in metrics_dict[algo].keys():
        logging.info(f'Expl mult: {expl_mult}')
        for metric in  metrics_dict[algo][expl_mult].keys():
            logging.info(f'Metric: {metric}')
            logging.info(f"mean: {np.mean(metrics_dict[algo][expl_mult][metric])} +- {np.std(metrics_dict[algo][expl_mult][metric])} ")
            logging.info(f"interval:  [{np.min(metrics_dict[algo][expl_mult][metric])}, {np.max(metrics_dict[algo][expl_mult][metric])}]")
            logging.info("============================================")
        # if 0 in metrics_dict[algo][expl_mult]:
        #     logging.info(f'rerun {metrics_dict[algo][expl_mult][metric].index(0)} with plotting')
    
    logging.info("============================================")
    

INFO:root:Algo: UCB
INFO:root:Expl mult: 1
INFO:root:Metric: jaccards
INFO:root:mean: 0.15909307875894987 +- 0.04360587601104567 
INFO:root:interval:  [0.04295942720763723, 0.2553699284009546]
INFO:root:============================================
INFO:root:Metric: percent_inter
INFO:root:mean: 1.0 +- 0.0 
INFO:root:interval:  [1.0, 1.0]
INFO:root:============================================
INFO:root:Metric: percent_found
INFO:root:mean: 0.15909307875894987 +- 0.04360587601104567 
INFO:root:interval:  [0.04295942720763723, 0.2553699284009546]
INFO:root:============================================
INFO:root:Metric: fails
INFO:root:mean: 0.0 +- 0.0 
INFO:root:interval:  [0, 0]
INFO:root:============================================
INFO:root:Expl mult: 10
INFO:root:Metric: jaccards
INFO:root:mean: 0.18434367541766108 +- 0.038143054432386254 
INFO:root:interval:  [0.06205250596658711, 0.25059665871121717]
INFO:root:============================================
INFO:root:Metric: percent_int

## Test avec dropout plutot que bonus d'exploration

In [17]:
metrics_dict = {}
algos = ["regTS"]
logging.info(metrics_dict)
max_n_steps = 10
for algo in algos:
    metrics_dict[algo] = {}
    for dropout_rate in [0.2, 0.5, 0.8]:
        metrics_dict[algo][str(dropout_rate)] = {}
        metrics_dict[algo][str(dropout_rate)]["jaccards"] = []
        metrics_dict[algo][str(dropout_rate)]["percent_inter"] = []
        metrics_dict[algo][str(dropout_rate)]["percent_found"] = []
        metrics_dict[algo][str(dropout_rate)]["fails"] = 0
        b = 0

        for i in range(b, 50):
            logging.info(f"at algo: {algo}, dropout: {dropout_rate}, run: {i}")
            make_deterministic(seed=i)
            vecs, rewards = gen_warmup_vecs_and_rewards(10)
            vecs, rewards = vecs.to(device), rewards.to(device)
            n_trials = 500
            width = 100
            net = NetworkDropout(d, width, dropout=dropout_rate).to(device)
            reg = 1
            sampletype = "f"
            reward_fn = reward_fn
            de_config = DEConfig
            de_policy = PullPolicy
            bern_p = 1 - dropout_rate
            p_vec = torch.tensor([bern_p] * width)

            agent = DENeuralTSDiag(
                net, lambda_=reg, style="ts", sampletype=sampletype
            )

            vecs, rewards = gen_warmup_vecs_and_rewards(10)
            vecs, rewards = vecs.to(device), rewards.to(device)
            agent.train_dataset.set_(vecs, rewards)

            # Warmup
            for vec in vecs:
                activ, grad = agent.compute_activation_and_grad(vec)
                agent.U += grad * grad

            agent.net.eval()
            agent.train(max_n_steps, patience=max_n_steps, lds=False)
            # Train
            for j in range(n_trials):
                agent.net.train()
                a_t, idx, best_member_grad = do_gradient_optim(agent, 60, x, lr=1e-2)
                agent.net.eval()
                r_t = reward_fn(a_t).unsqueeze(0).unsqueeze(0)

                if best_member_grad is None:
                    break
                agent.U += best_member_grad * best_member_grad

                agent.train_dataset.add(a_t, r_t)
                agent.train(max_n_steps, patience=max_n_steps, lds=False)

            if best_member_grad is None:
                metrics_dict[algo][str(dropout_rate)]["fails"] += 1
                logging.info(
                    f"Encountered a fail in {algo} {dropout_rate} because of nans"
                )
                continue
            agent.net.eval()
            sol, _, _ = agent.find_solution_in_vecs(x, 0, n_sigmas=n_sigmas)
            sol = sol.to(device)
            n_sol = len(sol)

            jaccard, n_inter = compute_jaccard(sol, true_sol)
            percent_found = n_inter / n_true_sol
            if n_sol == 0:
                percent_inter = 0
            else:
                percent_inter = n_inter / n_sol

            metrics_dict[algo][str(dropout_rate)]["jaccards"].append(jaccard)
            metrics_dict[algo][str(dropout_rate)]["percent_inter"].append(percent_inter)
            metrics_dict[algo][str(dropout_rate)]["percent_found"].append(percent_found)

            if n_sol == 0:
                logging.info(f"Found no solution for run {i}")
                metrics_dict[algo][str(dropout_rate)]["fails"] += 1

                plot_estimate(
                    agent,
                    n_trials,
                    fn=f"grad_no_sol_{algo}_drop_{dropout_rate}_100_trials_seed_{i}.png",
                )
            agent.net.train()

            logging.info(
                f"jaccard: {jaccard}, percent_inter: {percent_inter}, percent_found: {percent_found}"
            )

torch.save(metrics_dict, f"metrics_dict_exp_synth_{max_n_steps}_dropout.pth")


INFO:root:{}
INFO:root:at algo: regTS, dropout: 0.2, run: 0
INFO:root:jaccard: 0.30787589498806683, percent_inter: 1.0, percent_found: 0.30787589498806683
INFO:root:at algo: regTS, dropout: 0.2, run: 1
INFO:root:jaccard: 0.3269689737470167, percent_inter: 1.0, percent_found: 0.3269689737470167
INFO:root:at algo: regTS, dropout: 0.2, run: 2
INFO:root:jaccard: 0.324582338902148, percent_inter: 1.0, percent_found: 0.324582338902148
INFO:root:at algo: regTS, dropout: 0.2, run: 3
INFO:root:jaccard: 0.315035799522673, percent_inter: 1.0, percent_found: 0.315035799522673
INFO:root:at algo: regTS, dropout: 0.2, run: 4
INFO:root:jaccard: 0.315035799522673, percent_inter: 1.0, percent_found: 0.315035799522673
INFO:root:at algo: regTS, dropout: 0.2, run: 5
INFO:root:jaccard: 0.24105011933174225, percent_inter: 1.0, percent_found: 0.24105011933174225
INFO:root:at algo: regTS, dropout: 0.2, run: 6
INFO:root:jaccard: 0.2935560859188544, percent_inter: 1.0, percent_found: 0.2935560859188544
INFO:root

<Figure size 980x700 with 0 Axes>

In [18]:
for algo in metrics_dict.keys():
    logging.info(f"Algo: {algo}")
    for dropout_rate in metrics_dict[algo].keys():
        logging.info(f'dropout: {dropout_rate}')
        for metric in  metrics_dict[algo][dropout_rate].keys():
            logging.info(f'Metric: {metric}')
            logging.info(f"mean: {np.mean(metrics_dict[algo][dropout_rate][metric])} +- {np.std(metrics_dict[algo][dropout_rate][metric])} ")
            logging.info(f"interval:  [{np.min(metrics_dict[algo][dropout_rate][metric])}, {np.max(metrics_dict[algo][dropout_rate][metric])}]")
            logging.info("============================================")
        # if 0 in metrics_dict[algo][expl_mult]:
        #     logging.info(f'rerun {metrics_dict[algo][expl_mult][metric].index(0)} with plotting')
    
    logging.info("============================================")
    

INFO:root:Algo: regTS
INFO:root:dropout: 0.2
INFO:root:Metric: jaccards
INFO:root:mean: 0.2937947494033412 +- 0.03064294154891065 
INFO:root:interval:  [0.21479713603818615, 0.3556085918854415]
INFO:root:============================================
INFO:root:Metric: percent_inter
INFO:root:mean: 1.0 +- 0.0 
INFO:root:interval:  [1.0, 1.0]
INFO:root:============================================
INFO:root:Metric: percent_found
INFO:root:mean: 0.2937947494033412 +- 0.03064294154891065 
INFO:root:interval:  [0.21479713603818615, 0.3556085918854415]
INFO:root:============================================
INFO:root:Metric: fails
INFO:root:mean: 0.0 +- 0.0 
INFO:root:interval:  [0, 0]
INFO:root:============================================
INFO:root:dropout: 0.5
INFO:root:Metric: jaccards
INFO:root:mean: 0.32438570638404707 +- 0.053033704565701126 
INFO:root:interval:  [0.1575178997613365, 0.4152744630071599]
INFO:root:============================================
INFO:root:Metric: percent_inter


# TEST: Keep dropout active all the time instead of turning it off during training

In [19]:
metrics_dict = {}
algos = ["regTS"]
logging.info(metrics_dict)
max_n_steps = 10
for algo in algos:
    metrics_dict[algo] = {}
    for dropout_rate in [0.2, 0.5, 0.8]:
        metrics_dict[algo][str(dropout_rate)] = {}
        metrics_dict[algo][str(dropout_rate)]["jaccards"] = []
        metrics_dict[algo][str(dropout_rate)]["percent_inter"] = []
        metrics_dict[algo][str(dropout_rate)]["percent_found"] = []
        metrics_dict[algo][str(dropout_rate)]["fails"] = 0
        b = 0

        for i in range(b, 50):
            logging.info(f"at algo: {algo}, dropout: {dropout_rate}, run: {i}")
            make_deterministic(seed=i)
            vecs, rewards = gen_warmup_vecs_and_rewards(10)
            vecs, rewards = vecs.to(device), rewards.to(device)
            n_trials = 500
            width = 100
            net = NetworkDropout(d, width, dropout=dropout_rate).to(device)
            reg = 1
            sampletype = "f"
            reward_fn = reward_fn
            de_config = DEConfig
            de_policy = PullPolicy
            bern_p = 1 - dropout_rate
            p_vec = torch.tensor([bern_p] * width)

            agent = DENeuralTSDiag(
                net, lambda_=reg, style="ts", sampletype=sampletype
            )

            vecs, rewards = gen_warmup_vecs_and_rewards(10)
            vecs, rewards = vecs.to(device), rewards.to(device)
            agent.train_dataset.set_(vecs, rewards)

            # Warmup
            for vec in vecs:
                activ, grad = agent.compute_activation_and_grad(vec)
                agent.U += grad * grad

                # agent.net.eval()
            agent.train(max_n_steps, patience=max_n_steps, lds=False)
                # agent.net.train()

            # Train
            for j in range(n_trials):

                best_member = find_best_member(agent.get_sample, de_config=de_config, seed=j)
                best_member_grad = best_member.activation_grad
                a_t = best_member.point.unsqueeze(0)
                r_t = reward_fn(a_t).unsqueeze(0).unsqueeze(0)

                if best_member_grad is None:
                    break
                agent.U += best_member_grad * best_member_grad

                agent.train_dataset.add(a_t, r_t)


                # agent.net.eval()
                agent.train(max_n_steps, patience=max_n_steps, lds=False)
                # agent.net.train()

            if best_member_grad is None:
                metrics_dict[algo][str(dropout_rate)]["fails"] += 1
                logging.info(
                    f"Encountered a fail in {algo} {dropout_rate} because of nans"
                )
                continue
            agent.net.eval()
            sol, _, _ = agent.find_solution_in_vecs(x, 0, n_sigmas=n_sigmas)
            sol = sol.to(device)
            n_sol = len(sol)

            jaccard, n_inter = compute_jaccard(sol, true_sol)
            percent_found = n_inter / n_true_sol
            if n_sol == 0:
                percent_inter = 0
            else:
                percent_inter = n_inter / n_sol

            metrics_dict[algo][str(dropout_rate)]["jaccards"].append(jaccard)
            metrics_dict[algo][str(dropout_rate)]["percent_inter"].append(percent_inter)
            metrics_dict[algo][str(dropout_rate)]["percent_found"].append(percent_found)

            if n_sol == 0:
                logging.info(f"Found no solution for run {i}")
                metrics_dict[algo][str(dropout_rate)]["fails"] += 1

                plot_estimate(
                    agent,
                    n_trials,
                    fn=f"grad_no_sol_{algo}_drop_{dropout_rate}_100_trials_seed_{i}.png",
                )
            agent.net.train()

            logging.info(
                f"jaccard: {jaccard}, percent_inter: {percent_inter}, percent_found: {percent_found}"
            )

torch.save(metrics_dict, f"metrics_dict_exp_synth_{max_n_steps}_dropout.pth")


INFO:root:{}
INFO:root:at algo: regTS, dropout: 0.2, run: 0
INFO:root:jaccard: 0.37708830548926014, percent_inter: 1.0, percent_found: 0.37708830548926014
INFO:root:at algo: regTS, dropout: 0.2, run: 1
INFO:root:Found no solution for run 1
INFO:root:jaccard: 0.0, percent_inter: 0, percent_found: 0.0
INFO:root:at algo: regTS, dropout: 0.2, run: 2
INFO:root:Found no solution for run 2
INFO:root:jaccard: 0.0, percent_inter: 0, percent_found: 0.0
INFO:root:at algo: regTS, dropout: 0.2, run: 3
INFO:root:Found no solution for run 3
INFO:root:jaccard: 0.0, percent_inter: 0, percent_found: 0.0
INFO:root:at algo: regTS, dropout: 0.2, run: 4
INFO:root:jaccard: 0.317016317016317, percent_inter: 0.9315068493150684, percent_found: 0.324582338902148
INFO:root:at algo: regTS, dropout: 0.2, run: 5
INFO:root:Found no solution for run 5
INFO:root:jaccard: 0.0, percent_inter: 0, percent_found: 0.0
INFO:root:at algo: regTS, dropout: 0.2, run: 6
INFO:root:jaccard: 0.26014319809069214, percent_inter: 1.0, p

<Figure size 980x700 with 0 Axes>

In [20]:
for algo in metrics_dict.keys():
    logging.info(f"Algo: {algo}")
    for dropout_rate in metrics_dict[algo].keys():
        logging.info(f'dropout: {dropout_rate}')
        for metric in  metrics_dict[algo][dropout_rate].keys():
            logging.info(f'Metric: {metric}')
            logging.info(f"mean: {np.mean(metrics_dict[algo][dropout_rate][metric])} +- {np.std(metrics_dict[algo][dropout_rate][metric])} ")
            logging.info(f"interval:  [{np.min(metrics_dict[algo][dropout_rate][metric])}, {np.max(metrics_dict[algo][dropout_rate][metric])}]")
            logging.info("============================================")
        # if 0 in metrics_dict[algo][expl_mult]:
        #     logging.info(f'rerun {metrics_dict[algo][expl_mult][metric].index(0)} with plotting')
    
    logging.info("============================================")
    

INFO:root:Algo: regTS
INFO:root:dropout: 0.2
INFO:root:Metric: jaccards
INFO:root:mean: 0.1501720269006353 +- 0.16024916488459579 
INFO:root:interval:  [0.0, 0.4146919431279621]
INFO:root:============================================
INFO:root:Metric: percent_inter
INFO:root:mean: 0.4283756685499898 +- 0.45955056769861136 
INFO:root:interval:  [0.0, 1.0]
INFO:root:============================================
INFO:root:Metric: percent_found
INFO:root:mean: 0.15723150357995228 +- 0.16673451227207464 
INFO:root:interval:  [0.0, 0.41766109785202865]
INFO:root:============================================
INFO:root:Metric: fails
INFO:root:mean: 26.0 +- 0.0 
INFO:root:interval:  [26, 26]
INFO:root:============================================
INFO:root:dropout: 0.5
INFO:root:Metric: jaccards
INFO:root:mean: 0.17586304609232314 +- 0.17498813780732267 
INFO:root:interval:  [0.0, 0.4423529411764706]
INFO:root:============================================
INFO:root:Metric: percent_inter
INFO:root:me